In [1]:
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image

from DataLoader import CustomDataLoader
from UpNet import UpNet

In [2]:
model = UpNet()

In [3]:
# https://pytorch.org/tutorials/beginner/saving_loading_models.html
path = ""
path_state_dict = path+"checkpoints/state_dict.pth"
train_on_gpu = torch.cuda.is_available()


In [4]:
transform2Tensor = transforms.ToTensor()
transform2PIL = transforms.ToPILImage()
model.load_state_dict(torch.load(path_state_dict))
model.eval()

if train_on_gpu:
    model = model.cuda()

In [8]:
image_path = "res/this.jpg"

with torch.no_grad():
    image = Image.open(image_path)
    image_tensor = transform2Tensor(image)
    image_tensor = image_tensor.unsqueeze(0)

    if train_on_gpu:
        image_tensor = image_tensor.cuda()

    output = model(image_tensor)
    output = output.squeeze().cpu()


    image_trans = transform2PIL(output)
    image_trans.show()